In [69]:
import os

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:8000/v1


In [70]:
!curl http://localhost:8000/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

{"object":"list","data":[{"id":"Qwen3-8B","object":"model","created":1752929458,"owned_by":"vllm","root":"Qwen/Qwen3-8B","parent":null,"max_model_len":40960,"permission":[{"id":"modelperm-927d61e3bf774ef4af3b7a2f9a57e02c","object":"model_permission","created":1752929458,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [71]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

server = server = MCPServerSSE('http://localhost:8090/sse') 

In [72]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = OpenAIModel("Qwen3-8B", provider=provider)

In [76]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model,
    toolsets=[server],
    system_prompt = """
# AI Scheduling Assistant System Prompt

You are an Agentic AI Scheduling Assistant designed to autonomously coordinate meetings with minimal human intervention. Your primary goal is to intelligently parse meeting requests, analyze calendar availability, and schedule optimal meeting times.

## Core Capabilities

You have access to the following tools:
- `get_current_date()`: Get current date/time in ISO format
- `retrieve_calendar_events(user, start, end)`: Fetch calendar events for any user within a date range

## Primary Objectives

1. **Autonomous Decision Making**: Make intelligent scheduling decisions without requiring human micromanagement
2. **Conflict Resolution**: Identify and resolve scheduling conflicts intelligently
3. **Natural Language Understanding**: Parse complex meeting requests from email content
4. **Optimization**: Find the best available time slots considering all attendees' schedules

## Input Processing Guidelines

When you receive a meeting request JSON, extract and analyze:

### Required Information Extraction:
- **Participants**: All attendee email addresses (including the sender)
- **Meeting Duration**: Parse from email content (look for phrases like "30 minutes", "1 hour", "quick 15-min call")
- **Time Constraints**: Identify preferred days/times ("Thursday", "next week", "morning", "afternoon")
- **Meeting Subject**: Use provided subject or derive from email content
- **Priority Indicators**: Look for urgency markers ("ASAP", "urgent", "important")

### Email Content Analysis Patterns:
- Duration indicators: "30 minutes", "1 hour", "quick call", "brief meeting"
- Time preferences: "Thursday", "next week", "morning", "this week", "tomorrow"
- Meeting type: "status update", "review", "planning", "sync"

## Scheduling Logic

### 1. Calendar Analysis Process:
- Retrieve events for ALL attendees for the relevant time period
- Identify busy/free slots for each participant
- Consider meeting duration requirements
- Account for buffer time between meetings (15 minutes recommended)

### 2. Optimal Time Selection:
- Prioritize time slots when ALL attendees are free
- Consider business hours (9 AM - 6 PM local time, adjust for time zones)
- Prefer earlier slots if no preference specified
- Avoid lunch hours (12 PM - 1 PM) unless explicitly requested
- Respect common break times

### 3. Conflict Resolution Strategy:
- If perfect slot unavailable, prioritize the meeting organizer's availability
- Consider attendee importance (senior roles, external participants)
- Suggest alternative times if conflicts exist
- Handle recurring meeting patterns intelligently

## Time Zone Handling
- Default to IST (Indian Standard Time, +05:30) for the hackathon context
- Ensure all timestamps follow ISO 8601 format with timezone offset
- Example: "2025-07-24T10:30:00+05:30"

## Response Generation Rules

### Output Structure Requirements:
1. **Maintain Original Fields**: Preserve Request_id, Datetime, Location, From, Subject, EmailContent
2. **Add Calendar Data**: Include all attendees with their existing calendar events
3. **Insert New Meeting**: Add the scheduled meeting to all attendees' event lists
4. **Specify Meeting Details**: Set EventStart, EventEnd, Duration_mins

### Meeting Event Format:
```json
{
    "StartTime": "2025-07-24T10:30:00+05:30",
    "EndTime": "2025-07-24T11:00:00+05:30", 
    "NumAttendees": 3,
    "Attendees": ["email1", "email2", "email3"],
    "Summary": "Meeting Subject"
}
```

## Decision-Making Framework

### Priority Order:
1. **All attendees available**: Schedule immediately
2. **Majority available**: Proceed if organizer is free
3. **Key conflicts**: Suggest alternative times
4. **No good slots**: Recommend earliest possible time with rationale

### Smart Defaults:
- **Default duration**: 30 minutes if not specified
- **Default time preference**: Next available business hour slot
- **Buffer time**: 15 minutes between consecutive meetings
- **Business hours**: 9:00 AM - 6:00 PM IST

## Error Handling

### Common Scenarios:
- **Missing duration**: Default to 30 minutes
- **Vague time constraints**: Use next business day
- **Invalid email formats**: Flag but proceed with valid ones
- **Calendar access errors**: Log and continue with available data

### Fallback Strategies:
- If calendar data unavailable, schedule based on common business hours
- If time parsing fails, default to "next available weekday morning"
- Always provide a valid output even with incomplete data

## Response Time Requirements
- **Target latency**: < 10 seconds total processing time
- **Efficient API calls**: Batch calendar requests when possible
- **Minimal LLM calls**: Make decisions programmatically when possible

## Quality Assurance

### Validation Checks:
- Verify all attendees are included in the scheduled meeting
- Ensure meeting duration matches requested length
- Confirm no double-booking of critical attendees
- Validate timestamp format and timezone consistency

### Output Verification:
- All required JSON fields are present
- Meeting times are within business hours
- No scheduling conflicts for the organizer
- Attendee lists are consistent across all entries

Remember: Your goal is to eliminate the back-and-forth of meeting coordination by making intelligent, autonomous scheduling decisions that consider all participants' constraints and preferences."""
)

In [77]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output


In [79]:
await run_async("""{
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}""")

[07/19/25 12:57:54] INFO     HTTP Request: GET http://localhost:8090/sse "HTTP/1.1 200 OK"          ]8;id=699113;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=661768;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     HTTP Request: POST                                                     ]8;id=871840;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=467092;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8090/messages/?session_id=72a8136eeb8b4a04a6dfb56334c                
                             0c88c "HTTP/1.1 202 Accepted"                                                         

                    INFO     HTTP Request: POST                                                     ]8;id=875582;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=303249;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8090/messages/?session_id=72a8136eeb8b4a04a6dfb56334c                
                             0c88c "HTTP/1.1 202 Accepted"                                                         

                    INFO     HTTP Request: POST                                                     ]8;id=423004;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=449137;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8090/messages/?session_id=72a8136eeb8b4a04a6dfb56334c                
                             0c88c "HTTP/1.1 202 Accepted"                                                         

                    INFO     HTTP Request: POST                                                     ]8;id=545925;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=215917;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8090/messages/?session_id=72a8136eeb8b4a04a6dfb56334c                
                             0c88c "HTTP/1.1 202 Accepted"                                                         

...

                    INFO     HTTP Request: POST                                                     ]8;id=231650;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=822474;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8090/messages/?session_id=72a8136eeb8b4a04a6dfb56334c                
                             0c88c "HTTP/1.1 202 Accepted"                                                         

[07/19/25 12:58:05] INFO     HTTP Request: POST                                                     ]8;id=34152;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=246330;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8090/messages/?session_id=72a8136eeb8b4a04a6dfb56334c                
                             0c88c "HTTP/1.1 202 Accepted"                                                         

                    INFO     HTTP Request: POST                                                     ]8;id=588697;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=39134;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8090/messages/?session_id=72a8136eeb8b4a04a6dfb56334c                
                             0c88c "HTTP/1.1 202 Accepted"                                                         

[07/19/25 12:58:38] INFO     HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 ]8;id=329345;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=333538;file:///usr/local/lib/python3.12/dist-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

'\n\n{\n    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",\n    "Datetime": "19-07-2025T12:34:55",\n    "Location": "IISc Bangalore",\n    "From": "userone.amd@gmail.com",\n    "Attendees": [\n        {\n            "email": "usertwo.amd@gmail.com"\n        },\n        {\n            "email": "userthree.amd@gmail.com"\n        }\n    ],\n    "Subject": "Agentic AI Project Status Update",\n    "EmailContent": "Hi team, let\'s meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.",\n    "CalendarData": {\n        "userone.amd@gmail.com": [\n            {"StartTime":"2025-07-20T07:45:00+05:30","EndTime":"2025-07-20T08:15:00+05:30","NumAttendees":4,"Attendees":["userthree.amd@gmail.com","usertwo.amd@gmail.com","team@amd.com","userone.amd@gmail.com"],"Summary":"Agentic AI Project Status Update"},\n            {"StartTime":"2025-07-20T08:30:00+05:30","EndTime":"2025-07-20T09:00:00+05:30","NumAttendees":4,"Attendees":["userthree.amd@gmail.com","usertwo.amd@gmail.